# Week 4 Lab: Simple Linear Regression (OLS)

**QM 2023 -- Statistics II / Data Analytics**  
**Spring 2026 | University of Tulsa**

---

## What You Will Learn

By the end of this lab, you will be able to:

1. **Fit** a simple OLS regression in Python using `statsmodels`
2. **Read** the regression output table and identify key statistics
3. **Interpret** the slope coefficient in economic units
4. **Assess** statistical significance using t-statistics, p-values, and confidence intervals
5. **Visualize** a regression relationship with a scatter plot and fitted line
6. **Evaluate** model fit using R-squared

---

### The Big Question

> *Do larger REITs earn different returns than smaller REITs?*

We will answer this by estimating:

$$\text{ret} = \beta_0 + \beta_1 \times \text{mcap} + \varepsilon$$

where:
- `ret` = monthly return for an individual REIT (decimal, e.g., 0.05 = 5%)
- `mcap` = market capitalization in millions of dollars
- $\beta_1$ = the **marginal effect** of size on returns

---

## Part 1: Setup and Data Loading

First, let's import the libraries we need and load our dataset.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# The key package for regression
from statsmodels.formula.api import ols

# Make plots look clean
sns.set_theme(style="whitegrid", context="notebook")

print("All imports successful!")

In [ ]:
# Load the REIT master data
df = pd.read_csv("data/reit_master_sample.csv")

# Drop rows with missing ret or mcap
df = df.dropna(subset=["ret", "mcap"])

print(f"Dataset shape: {df.shape[0]} rows x {df.shape[1]} columns")
print(f"Unique REITs: {df['permno'].nunique()}")
print()
df.head()

### Understanding the Variables

| Variable | Meaning | Example |
|:---------|:--------|:--------|
| `permno` | Unique REIT identifier | 10001 |
| `ym` | Year-month | 202101 = Jan 2021 |
| `ret` | Monthly return | 0.052 = 5.2% |
| `mcap` | Market cap (millions $) | 5200 |
| `sector` | Property sector | Retail, Office, Industrial |
| `price` | Stock price | 45.30 |

**Important:** Returns are in decimal form. 0.01 = 1 percentage point. Market cap is in millions.

In [ ]:
# Quick summary statistics for the two variables we care about
print("Summary Statistics")
print("=" * 50)
df[["ret", "mcap"]].describe().round(4)

#### Checkpoint 1

Before we model anything, look at the summary statistics above.

- What is the average monthly return across REITs?
- What is the range of market cap (smallest to largest)?
- Do returns have negative values? What does that mean economically?

---

## Part 2: Looking at the Data First (Always!)

Before running any regression, **always visualize your data**. A scatter plot lets us see whether a linear relationship is plausible.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.scatter(df["mcap"], df["ret"],
           color="#2980B9", alpha=0.7, s=60,
           edgecolor="black", linewidth=0.4)

ax.set_xlabel("Market Cap (millions $)", fontsize=12)
ax.set_ylabel("Monthly Return", fontsize=12)
ax.set_title("Do larger REITs earn different returns?", fontsize=13, fontweight="bold")
ax.grid(True, alpha=0.2, linestyle="--")
ax.axhline(0, color="gray", linewidth=0.8, linestyle="-")

plt.tight_layout()
plt.show()

#### Checkpoint 2

Look at the scatter plot above.

- Does the relationship look **linear**? (Could a straight line summarize the pattern?)
- Is the relationship **positive or negative**? (Do larger REITs tend to have higher or lower returns?)
- Are there any obvious **outliers**? (Points far from the cloud?)

If the answer to the first question is "yes," OLS regression is a reasonable tool to use.

---

## Part 3: Fitting Your First OLS Regression

OLS stands for **Ordinary Least Squares**. It finds the straight line that minimizes the total squared distance between each data point and the line.

In Python, we use `statsmodels` with an R-style formula:

```python
model = ols('Y ~ X', data=df).fit()
```

The `~` means "is predicted by." So `ret ~ mcap` means:  
*"Monthly return is predicted by market capitalization."*

In [ ]:
# Fit the OLS regression
model = ols("ret ~ mcap", data=df).fit()

# Print the full output table
print(model.summary())

---

## Part 4: Reading the Regression Output

That output table has a lot of numbers. Let's break it down into three sections.

### Section 1: Top Panel (Model Info)

| What to look for | What it tells you |
|:-----------------|:------------------|
| **Dep. Variable** | What we're predicting (`ret`) |
| **R-squared** | How much variance the model explains (0 to 1) |
| **No. Observations** | Sample size |

### Section 2: Coefficient Table (The Heart of the Output)

| Column | Meaning |
|:-------|:--------|
| **coef** | The estimated value of the coefficient |
| **std err** | How precise our estimate is (smaller = more precise) |
| **t** | Test statistic: `coef / std err` |
| **P>\|t\|** | p-value: probability of seeing this result if the true effect were zero |
| **[0.025, 0.975]** | 95% confidence interval for the coefficient |

### Section 3: Bottom Panel (Diagnostics)

We'll cover these in detail in **Week 6**. For now, just note they exist.

---

## Part 5: Extracting Key Results

Rather than reading numbers off a big table, we can extract them directly from the model object. This is useful for reports and for checking your work.

In [ ]:
# ---- Coefficients ----
intercept = model.params["Intercept"]
slope = model.params["mcap"]

print("COEFFICIENTS")
print(f"  Intercept (B0): {intercept:.6f}")
print(f"  Slope (B1):     {slope:.6f}")
print()
print(f"  Our estimated equation:")
print(f"  ret = {intercept:.4f} + {slope:.6f} * mcap")

In [ ]:
# ---- Standard Errors ----
se_intercept = model.bse["Intercept"]
se_slope = model.bse["mcap"]

print("STANDARD ERRORS (precision of our estimates)")
print(f"  SE(Intercept): {se_intercept:.6f}")
print(f"  SE(Slope):     {se_slope:.6f}")
print()
print("  Smaller SE = more precise estimate")

In [ ]:
# ---- t-statistics and p-values ----
t_slope = model.tvalues["mcap"]
p_slope = model.pvalues["mcap"]

print("HYPOTHESIS TEST: Is the slope different from zero?")
print(f"  t-statistic: {t_slope:.2f}")
print(f"  p-value:     {p_slope:.6f}")
print()

# Interpret
if p_slope < 0.001:
    print("  Result: p < 0.001 -- VERY strong evidence of a relationship")
elif p_slope < 0.05:
    print(f"  Result: p = {p_slope:.4f} -- Statistically significant at 5% level")
else:
    print(f"  Result: p = {p_slope:.4f} -- NOT statistically significant at 5% level")

In [ ]:
# ---- Confidence Interval ----
ci = model.conf_int().loc["mcap"]
ci_low, ci_high = ci[0], ci[1]

print("95% CONFIDENCE INTERVAL for the slope")
print(f"  [{ci_low:.6f}, {ci_high:.6f}]")
print()
print(f"  We are 95% confident the true marginal effect of mcap is between {ci_low:.6f} and {ci_high:.6f}.")
print(f"  Does this interval contain 0? {'Yes -- not significant' if ci_low <= 0 <= ci_high else 'No -- significant!'}")

In [ ]:
# ---- R-squared ----
r2 = model.rsquared
r2_adj = model.rsquared_adj

print("MODEL FIT")
print(f"  R-squared:          {r2:.4f}")
print(f"  Adjusted R-squared: {r2_adj:.4f}")
print()
print(f"  {r2*100:.1f}% of the variation in returns")
print(f"  is explained by market cap.")
print(f"  The remaining {(1-r2)*100:.1f}% is unexplained (other factors, noise).")

#### Checkpoint 3

Using the results above, answer these questions (write your answers in the cell below):

1. What is the slope coefficient? What does it mean *in plain English*?
2. Is the slope statistically significant? How do you know?
3. What percentage of variation in returns is explained by the model?
4. Does the 95% confidence interval contain zero?

*Your answers here:*

1. ...
2. ...
3. ...
4. ...

---

## Part 6: Writing a Proper Interpretation

In this class, **numbers without words are incomplete**. Here is how a professional analyst would interpret these results.

### The Wrong Way

> "The slope is 0.62 and the p-value is less than 0.05."

This is technically correct but tells the reader nothing useful.

### The Right Way

> "A $1 million increase in market cap is associated with a [slope] change in monthly return (beta1 = [value], t = [t], p = [p]). This suggests that [larger/smaller] REITs earn [higher/lower] returns on average. The model explains [X]% of the variation in returns (R-squared = [value]), indicating that size is [an important / a weak] predictor. Other factors such as sector and momentum likely matter as well."

### What Makes This Good?

- **Units are explicit:** "$1 million increase in market cap"
- **Evidence is cited:** t-stat, p-value, R-squared
- **Economic context:** what does the slope *mean* for investors?
- **Limitations acknowledged:** "other factors likely matter"

---

## Part 7: Visualizing the Regression

A scatter plot with the fitted regression line is the standard way to communicate a bivariate relationship. Let's build one step by step.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))

# 1. Scatter plot of the raw data
ax.scatter(df["mcap"], df["ret"],
           color="#2980B9", alpha=0.7, s=70,
           edgecolor="black", linewidth=0.4,
           label=f"REIT observations (n={len(df)})")

# 2. Fitted regression line
x_range = np.linspace(df["mcap"].min() - 500,
                      df["mcap"].max() + 500, 100)
y_hat = intercept + slope * x_range
ax.plot(x_range, y_hat, color="#C0392B", linewidth=2.5,
        label=f"Fitted line: y = {intercept:.4f} + {slope:.6f}x")

# 3. Labels and formatting
ax.set_xlabel("Market Cap (millions $)", fontsize=12)
ax.set_ylabel("Monthly Return", fontsize=12)
ax.set_title("REIT Monthly Return vs. Market Capitalization",
             fontsize=13, fontweight="bold")
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, alpha=0.2, linestyle="--")

# 4. R-squared annotation box
ax.text(0.97, 0.05,
        f"R$^2$ = {r2:.3f}\n({r2*100:.1f}% explained)",
        transform=ax.transAxes, fontsize=11,
        ha="right", va="bottom",
        bbox=dict(boxstyle="round,pad=0.4", facecolor="wheat",
                  edgecolor="#1F4E79", alpha=0.8))

plt.tight_layout()
plt.show()

---

## Part 8: Understanding Residuals

**Residuals** are the vertical distances between each data point and the regression line:

$$\text{residual}_i = y_i - \hat{y}_i = \text{actual} - \text{predicted}$$

OLS finds the line that makes the **sum of squared residuals** as small as possible.

Let's visualize what residuals look like.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))

# Pick 10 random points for clarity
np.random.seed(42)
idx = np.sort(np.random.choice(len(df), size=min(10, len(df)), replace=False))
df_sub = df.iloc[idx]
fitted_sub = model.fittedvalues.iloc[idx]

# All points (faded)
ax.scatter(df["mcap"], df["ret"],
           color="#2980B9", alpha=0.15, s=50)

# Highlighted subset
ax.scatter(df_sub["mcap"], df_sub["ret"],
           color="#2980B9", alpha=0.9, s=80,
           edgecolor="black", linewidth=0.6,
           label="Observed data points")

# Regression line
ax.plot(x_range, y_hat, color="#C0392B", linewidth=2.5, label="Fitted line")

# Draw residual lines (vertical dashed orange lines)
for i in range(len(df_sub)):
    x_val = df_sub["mcap"].iloc[i]
    y_obs = df_sub["ret"].iloc[i]
    y_fit = fitted_sub.iloc[i]
    ax.plot([x_val, x_val], [y_obs, y_fit],
            color="#E67E22", linewidth=1.5, linestyle="--",
            label="Residuals" if i == 0 else None)

ax.set_title("OLS Minimizes the Sum of Squared Residuals",
             fontweight="bold", fontsize=13)
ax.set_xlabel("Market Cap (millions $)", fontsize=12)
ax.set_ylabel("Monthly Return", fontsize=12)
ax.legend(loc="upper left", fontsize=10)
ax.grid(True, alpha=0.2, linestyle="--")

plt.tight_layout()
plt.show()

The **orange dashed lines** are the residuals. OLS chose the red line specifically because it makes the total area of those squared residual distances as small as possible.

In Week 6, we will check whether these residuals satisfy the **Gauss-Markov assumptions** (constant variance, normality, independence). For now, let's just look at their distribution.

In [ ]:
from scipy import stats

residuals = model.resid

fig, axes = plt.subplots(1, 2, figsize=(12, 4.5))

# Left: histogram of residuals
axes[0].hist(residuals, bins=12, color="#2980B9", alpha=0.7,
             edgecolor="white", density=True)
x_norm = np.linspace(residuals.min() - 0.002, residuals.max() + 0.002, 200)
axes[0].plot(x_norm,
             stats.norm.pdf(x_norm, loc=residuals.mean(), scale=residuals.std()),
             color="#C0392B", linewidth=2, label="Normal curve")
axes[0].axvline(0, color="#333", linestyle="--", linewidth=1, alpha=0.5)
axes[0].set_title("Distribution of Residuals", fontweight="bold")
axes[0].set_xlabel("Residual")
axes[0].set_ylabel("Density")
axes[0].legend()

# Right: residuals vs fitted values
axes[1].scatter(model.fittedvalues, residuals,
                color="#2980B9", alpha=0.6, s=50, edgecolor="black", linewidth=0.3)
axes[1].axhline(0, color="#C0392B", linewidth=1.5, linestyle="--")
axes[1].set_title("Residuals vs. Fitted Values", fontweight="bold")
axes[1].set_xlabel("Fitted values ($\\hat{y}$)")
axes[1].set_ylabel("Residuals")

plt.tight_layout()
plt.show()

print(f"Mean of residuals: {residuals.mean():.8f}  (should be ~0)")
print(f"Std of residuals:  {residuals.std():.6f}")

#### What to Look For

- **Left plot (histogram):** Do residuals look roughly normal (bell-shaped)? Good sign.
- **Right plot (residuals vs. fitted):** Do residuals spread evenly around zero, with no funnel shape or pattern? If yes, the **constant variance** assumption is plausible.

We will formally test these assumptions in **Week 6**.

---

## Part 9: Confidence Interval Visualization

The 95% confidence interval tells us the range of plausible values for the true slope. Let's visualize it.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2.5))

# Draw the CI bar
ax.plot([ci_low, ci_high], [0, 0], color="#1F4E79", linewidth=5,
        solid_capstyle="round")
ax.plot(ci_low, 0, "|", color="#1F4E79", markersize=20, markeredgewidth=3)
ax.plot(ci_high, 0, "|", color="#1F4E79", markersize=20, markeredgewidth=3)

# Point estimate
ax.plot(slope, 0, "o", color="#C0392B", markersize=14,
        markeredgecolor="white", markeredgewidth=2)

# Labels
ax.text(slope, 0.18, f"B1 = {slope:.4f}", ha="center", fontsize=13,
        fontweight="bold", color="#C0392B")
ax.text(ci_low, -0.18, f"{ci_low:.4f}", ha="center", fontsize=11, color="#1F4E79")
ax.text(ci_high, -0.18, f"{ci_high:.4f}", ha="center", fontsize=11, color="#1F4E79")

# H0 reference
ax.axvline(0, color="#999", linestyle="--", linewidth=1)
ax.text(0, 0.28, "H0: B1 = 0", ha="center", fontsize=10,
        color="#999", fontstyle="italic")

# Formatting
ax.set_xlim(min(ci_low - 0.1, -0.05), ci_high + 0.1)
ax.set_ylim(-0.4, 0.4)
ax.set_xlabel("Slope Coefficient (B1)", fontsize=12)
ax.set_yticks([])
for spine in ["top", "right", "left"]:
    ax.spines[spine].set_visible(False)
ax.set_title("95% Confidence Interval for the Slope", fontweight="bold")

plt.tight_layout()
plt.show()

print(f"The null hypothesis (B1 = 0) is {'inside' if ci_low <= 0 <= ci_high else 'outside'} the 95% CI.")
print(f"Therefore, we {'fail to reject' if ci_low <= 0 <= ci_high else 'reject'} H0 at the 5% significance level.")

---

## Part 10: Thinking About What We're Missing

Our model has one predictor. In reality, many things affect individual REIT returns:

| Omitted Variable | Why It Might Matter |
|:-----------------|:--------------------|
| Sector | Different property sectors (Retail, Office, Industrial) have different return patterns |
| Interest rates | REITs are rate-sensitive; rates affect valuations and returns |
| Momentum | Past returns may predict future returns |
| Value (e.g., P/FFO) | Valuation ratios may be related to both size and returns |
| Volatility | Riskier REITs may earn different returns |

If any of these are correlated with **both** `mcap` and `ret`, our slope estimate could be **biased**. This is called **omitted variable bias**.

We will address this in **Week 5** by adding multiple predictors to the model.

---

## Part 11: Your Turn -- Practice Exercises

Try these exercises on your own. They will help you prepare for **Assignment 04** (due Friday).

### Exercise 1: Try Log(Mcap)

Instead of `ret ~ mcap`, try regressing returns on the **log of market cap**:

```python
df["log_mcap"] = np.log(df["mcap"])
model2 = ols("ret ~ log_mcap", data=df).fit()
```

- Is the slope positive or negative?
- Is it statistically significant?
- How does R-squared compare to the level model? (Interpretation: log(mcap) = "a 1% increase in mcap")

In [ ]:
# Exercise 1: Your code here



### Exercise 2: By Sector

Create a subset of the data for one sector (e.g., Retail) and run the same regression:

```python
df_retail = df[df["sector"] == "Retail"]
model3 = ols("ret ~ mcap", data=df_retail).fit()
```

- Does the slope change compared to the full sample?
- Is the relationship stronger or weaker within one sector?
- Why might sector-specific results differ?

In [ ]:
# Exercise 2: Your code here



### Exercise 3: Create a Scatter Plot

Pick either Exercise 1 or 2 and create a scatter plot with the fitted regression line (like Part 7).

Make sure your plot has:
- A descriptive title
- Axis labels with the variable names
- A legend
- An R-squared annotation

In [ ]:
# Exercise 3: Your code here



### Exercise 4: Write an Interpretation

In the markdown cell below, write a 2--3 sentence interpretation of the model you estimated in Exercise 1 or 2. Follow the template from Part 6:

1. State the slope, what it means in economic units
2. State the statistical significance (cite t-stat and p-value)
3. State the R-squared and what it implies

*Your interpretation here:*

...

---

## Summary

| Concept | Key Takeaway |
|:--------|:-------------|
| **OLS** | Finds the line minimizing squared residuals |
| **Slope ($\beta_1$)** | Change in Y per 1-unit change in X |
| **Intercept ($\beta_0$)** | Predicted Y when X = 0 |
| **t-statistic** | `coef / std err` -- tests if coefficient differs from zero |
| **p-value** | Probability of observing this result if true effect = 0 |
| **95% CI** | Range of plausible values for the true coefficient |
| **R-squared** | Proportion of variance in Y explained by X (0 to 1) |
| **Residuals** | Actual - Predicted; should be centered at zero |

---

## What's Next

- **Friday:** Complete **Assignment 04** -- estimate `ret ~ mcap`, create a scatter plot, and write an interpretation memo. Due at 11:59 PM.
- **Week 5:** Multiple regression -- add sector, momentum, or other predictors to explain what our simple model misses.
- **Week 6:** Regression diagnostics -- formally test the Gauss-Markov assumptions.

---

*QM 2023 | Statistics II / Data Analytics | Spring 2026*